---
title: ""
---

In [ ]:
#| echo: false
#| output: asis

import pandas as pd

from cnvsapi.build_utils import get_current_week_from_lecture_id, get_outline_objects, extract_outline
from datetime import datetime, timedelta

output_file = "./modules-gantt.mmd"


outline_df = extract_outline().copy()
outline_df.loc[:,'date'] = pd.to_datetime(outline_df['DateCanvas']).dt.date

summary_df = outline_df.groupby('Module').agg(
    earliest_date=('date', 'min'),
    latest_date=('date', 'max')
).reset_index()
summary_df = summary_df.sort_values(by='earliest_date').reset_index(drop=True)

# Find the row(s) where "Notes, Lectures" contains "Classes begin"
matching_rows = outline_df[outline_df['Notes, Lectures'].str.lower().str.contains('classes begin', na=False)]
first_date = pd.to_datetime(matching_rows['date'].iloc[0]).date()  - timedelta(days=1)
if first_date is not None:
    summary_df.at[0, 'earliest_date'] = first_date

matching_rows = outline_df[outline_df['Notes, Lectures'].str.lower().str.contains('last day of classes', na=False)]
last_date = pd.to_datetime(matching_rows['date'].iloc[0]).date()
if last_date is not None:
    summary_df.at[summary_df.index[-1], 'latest_date'] = last_date
# Add one day to the latest dates to account for the 24-hours of the last day
summary_df['latest_date'] = summary_df['latest_date'].apply(lambda x: x + timedelta(days=1))


In [ ]:
def make_gantt():
    """ make gant chart """

    #original_date_str = "1/7/2024"
    #original_date = datetime.strptime(original_date_str, "%m/%d/%Y")
    #new_date = original_date + timedelta(weeks=(week_id - 1) )
    #new_date_str = new_date.strftime("%m/%d/%Y").lstrip("0").replace("/0", "/")


    # VCU gold: #FFB300

    # Define the content you want to write to the file
    content = []
    content.append(f"""gantt
    dateFormat YYYY-MM-DD
    axisFormat %b %e
    #todayMarker stroke-width:7px,stroke:#FFB300,opacity:0.5
"""
    )
    content.append('    section Modules')
    for i,item in summary_df.iterrows():
        content.append(f"    {item['Module']} : m{i}, {item['earliest_date']}, {item['latest_date']}")

    # Open the file in write mode ('w' will create the file if it doesn't exist or overwrite it if it does)
    with open(output_file, "w") as file:
        file.write("\n".join(content))

make_gantt()

::: {.no-filter}

```{mermaid}
%%| echo: false
%%| file: ./modules-gantt.mmd
```

:::